### 전체 지하철역 데이터 전처리(9호선은 없는듯 함)

In [1]:
import warnings
warnings.filterwarnings(action='ignore')
import scipy as sp
import scipy.stats
import statsmodels.api as sm
import sklearn as sk
import seaborn as sns
import matplotlib as mpl
import matplotlib.pylab as plt
import glob
set(sorted([f.name for f in mpl.font_manager.fontManager.ttflist]))
mpl.rc('font', family='NanumGothic')
mpl.rc('axes', unicode_minus=False)

In [2]:
path ='C:/Code/regression'
filenames = glob.glob(path + "/*.csv")

In [3]:
sub_2015=pd.DataFrame(pd.read_csv(filenames[0],encoding='euc-kr',thousands = ','))
sub_2016=pd.DataFrame(pd.read_csv(filenames[1],encoding='euc-kr',thousands = ','))
sub_2017=pd.DataFrame(pd.read_csv(filenames[2],encoding='euc-kr',thousands = ','))
sub_2018=pd.DataFrame(pd.read_csv(filenames[3],encoding='euc-kr',thousands = ','))
sub_2019=pd.DataFrame(pd.read_csv(filenames[4],encoding='euc-kr',thousands = ','))

In [4]:
sub_2015.columns=['날짜', '역번호', '역명', '구분', '7to8', '8to9', '9to10', '17to18', '18to19','19to20']
sub_2016.columns=['날짜', '역번호', '역명', '구분', '7to8', '8to9', '9to10', '17to18', '18to19','19to20']
sub_2017.columns=['날짜', '역번호', '역명', '구분', '7to8', '8to9', '9to10', '17to18', '18to19','19to20']
sub_2018.columns=['날짜', '역번호', '역명', '구분', '7to8', '8to9', '9to10', '17to18', '18to19','19to20']
sub_2019.columns=['날짜', '역번호', '역명', '구분', '7to8', '8to9', '9to10', '17to18', '18to19','19to20']

In [5]:
sub=pd.concat([sub_2015,sub_2016,sub_2017,sub_2018,sub_2019])

In [6]:
sub

,날짜,역번호,역명,구분,7to8,8to9,9to10,17to18,18to19,19to20
0,2015-01-01,150,서울역(150),승차,572,1106,1474,3666,3738,3091
1,2015-01-01,151,시청(151),승차,104,127,233,1176,1133,850
2,2015-01-01,152,종각(152),승차,281,291,328,2176,2047,1767
3,2015-01-01,153,종로3가(153),승차,155,189,347,1954,1526,1297
4,2015-01-01,154,종로5가(154),승차,83,128,209,948,962,722
...,...,...,...,...,...,...,...,...,...,...
200799,2019-12-31,2825,신흥,하차,86,243,208,510,562,493
200800,2019-12-31,2826,수진,승차,542,813,436,436,395,245
200801,2019-12-31,2826,수진,하차,115,433,259,463,501,452
200802,2019-12-31,2827,모란,승차,342,482,253,375,418,242


In [7]:
# 행 개수 확인
len(sub_2015)+len(sub_2016)+len(sub_2017)+len(sub_2018)+len(sub_2019)

1004354

In [8]:
# index 재할당
sub.reset_index(inplace=True,drop=True)

In [9]:
# to_numeric
sub[['역번호', '7to8', '8to9', '9to10', '17to18', '18to19','19to20']]=sub[['역번호', '7to8', '8to9', '9to10', '17to18', '18to19','19to20']].apply(pd.to_numeric)

In [10]:
sub.날짜=pd.to_datetime(sub['날짜'])

In [11]:
sub['year']=sub.날짜.apply(lambda x: x.year)
sub['month']=sub.날짜.apply(lambda x: x.month)

In [12]:
# 요일 찾기
import datetime
from datetime import date

t=['월','화','수','목','금','토','일']
sub["요일"]=sub['날짜'].apply(lambda x: t[x.weekday()])

In [13]:
sub

,날짜,역번호,역명,구분,7to8,8to9,9to10,17to18,18to19,19to20,year,month,요일
0,2015-01-01,150,서울역(150),승차,572,1106,1474,3666,3738,3091,2015,1,목
1,2015-01-01,151,시청(151),승차,104,127,233,1176,1133,850,2015,1,목
2,2015-01-01,152,종각(152),승차,281,291,328,2176,2047,1767,2015,1,목
3,2015-01-01,153,종로3가(153),승차,155,189,347,1954,1526,1297,2015,1,목
4,2015-01-01,154,종로5가(154),승차,83,128,209,948,962,722,2015,1,목
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1004349,2019-12-31,2825,신흥,하차,86,243,208,510,562,493,2019,12,화
1004350,2019-12-31,2826,수진,승차,542,813,436,436,395,245,2019,12,화
1004351,2019-12-31,2826,수진,하차,115,433,259,463,501,452,2019,12,화
1004352,2019-12-31,2827,모란,승차,342,482,253,375,418,242,2019,12,화


```




```

### 2호선 퇴근시간만

In [23]:
# 2호선 퇴근시간만
sub2=sub[(200<sub["역번호"]) & (sub["역번호"]<300)].reset_index(drop=True)
sub2.drop(['7to8', '8to9', '9to10'],1,inplace=True)

In [24]:
sub2

,날짜,역번호,역명,구분,17to18,18to19,19to20,year,month,요일
0,2015-01-01,201,시청(201),승차,889,860,632,2015,1,목
1,2015-01-01,202,을지로입구(202),승차,2079,2456,2063,2015,1,목
2,2015-01-01,203,을지로3가(203),승차,450,371,329,2015,1,목
3,2015-01-01,204,을지로4가(204),승차,284,206,175,2015,1,목
4,2015-01-01,205,동대문역사문화공원(205),승차,1006,980,909,2015,1,목
...,...,...,...,...,...,...,...,...,...,...
182595,2019-12-31,248,양천구청,하차,761,989,843,2019,12,화
182596,2019-12-31,249,신정네거리,승차,501,544,343,2019,12,화
182597,2019-12-31,249,신정네거리,하차,1011,1454,881,2019,12,화
182598,2019-12-31,250,용두(동대문구청),승차,253,252,162,2019,12,화


In [28]:
# 17~20시간대의 탑승객
sub2["17to20"]=sub2["17to18"]+sub2["18to19"]+sub2["19to20"]

In [27]:
# 역명에서 역번호 제외
sub2["역명"]=sub2.역명.str.split("(",expand=True)[0]

In [34]:
# 왜 위에꺼는 [0]하면 나오는데 이건 안나옴?????????????????????
# 둘다 똑같이 dataframe인데..
# A.위는 0이 컬럼명이었음..
sub2[0]

KeyError: 0

In [225]:
sub2.drop(["역번호","17to18","18to19","19to20"],1,inplace=True)

In [226]:
sub2

,날짜,역명,구분,year,month,요일,17to20
0,2015-01-01,시청,승차,2015,1,목,2381
1,2015-01-01,을지로입구,승차,2015,1,목,6598
2,2015-01-01,을지로3가,승차,2015,1,목,1150
3,2015-01-01,을지로4가,승차,2015,1,목,665
4,2015-01-01,동대문역사문화공원,승차,2015,1,목,2895
...,...,...,...,...,...,...,...
182595,2019-12-31,양천구청,하차,2019,12,화,2593
182596,2019-12-31,신정네거리,승차,2019,12,화,1388
182597,2019-12-31,신정네거리,하차,2019,12,화,3346
182598,2019-12-31,용두,승차,2019,12,화,667


In [227]:
len(sub2.역명.unique()),sub2.역명.unique()

(51,
 array(['시청', '을지로입구', '을지로3가', '을지로4가', '동대문역사문화공원', '신당', '상왕십리', '왕십리',
        '한양대', '뚝섬', '성수', '건대입구', '구의', '강변', '잠실나루', '잠실', '신천', '종합운동장',
        '삼성', '선릉', '역삼', '강남', '교대', '서초', '방배', '사당', '낙성대', '서울대입구',
        '봉천', '신림', '신대방', '구로디지털단지', '대림', '신도림', '문래', '영등포구청', '당산',
        '합정', '홍대입구', '신촌', '이대', '아현', '충정로', '용답', '신답', '신설동', '도림천',
        '양천구청', '신정네거리', '용두', '잠실새내'], dtype=object))

In [228]:
# 신천이 잠실새내로 변경되어 두개의 이름 존재
sub2.loc[sub2['역명']=="신천",'역명']="잠실새내"

In [229]:
sub2

,날짜,역명,구분,year,month,요일,17to20
0,2015-01-01,시청,승차,2015,1,목,2381
1,2015-01-01,을지로입구,승차,2015,1,목,6598
2,2015-01-01,을지로3가,승차,2015,1,목,1150
3,2015-01-01,을지로4가,승차,2015,1,목,665
4,2015-01-01,동대문역사문화공원,승차,2015,1,목,2895
...,...,...,...,...,...,...,...
182595,2019-12-31,양천구청,하차,2019,12,화,2593
182596,2019-12-31,신정네거리,승차,2019,12,화,1388
182597,2019-12-31,신정네거리,하차,2019,12,화,3346
182598,2019-12-31,용두,승차,2019,12,화,667


In [230]:
len(sub2.역명.unique()),sub2.역명.unique()

(50,
 array(['시청', '을지로입구', '을지로3가', '을지로4가', '동대문역사문화공원', '신당', '상왕십리', '왕십리',
        '한양대', '뚝섬', '성수', '건대입구', '구의', '강변', '잠실나루', '잠실', '잠실새내',
        '종합운동장', '삼성', '선릉', '역삼', '강남', '교대', '서초', '방배', '사당', '낙성대',
        '서울대입구', '봉천', '신림', '신대방', '구로디지털단지', '대림', '신도림', '문래', '영등포구청',
        '당산', '합정', '홍대입구', '신촌', '이대', '아현', '충정로', '용답', '신답', '신설동',
        '도림천', '양천구청', '신정네거리', '용두'], dtype=object))

In [231]:
np.sum(sub2.isnull())

날짜        0
역명        0
구분        0
year      0
month     0
요일        0
17to20    0
dtype: int64

### 2호선 데이터 공휴일 여부 표시

In [232]:
holi=pd.read_csv("holi.csv",encoding="euc-kr")

In [233]:
holi

,locdate,seq,isHoliday,dateName,year,week
0,20150101,1,Y,신정,2015,목
1,20150218,1,Y,설날,2015,수
2,20150219,1,Y,설날,2015,목
3,20150220,1,Y,설날,2015,금
4,20150301,1,Y,삼일절,2015,일
...,...,...,...,...,...,...
86,20190913,1,Y,추석,2019,금
87,20190914,1,Y,추석,2019,토
88,20191003,1,Y,개천절,2019,목
89,20191009,1,Y,한글날,2019,수


In [234]:
# seq 공휴일이 겹친경우 카운트 됨
holi[holi.seq==2].locdate

47    20171002
49    20171003
Name: locdate, dtype: int64

In [235]:
# 띠용 20171002는 seq 2 인데 한개 밖에 없눈데..
holi[(holi.locdate==20171002) | (holi.locdate==20171003)]

,locdate,seq,isHoliday,dateName,year,week
47,20171002,2,Y,임시공휴일,2017,월
48,20171003,1,Y,개천절,2017,화
49,20171003,2,Y,추석,2017,화


In [236]:
# isHoliday가 N인경우는 쉬는날이 아니었으므로 제거
holi=holi[holi.isHoliday!="N"]

In [237]:
holi.groupby("year").size()

year
2015    16
2016    17
2017    19
2018    18
2019    16
dtype: int64

In [238]:
holi.locdate=pd.to_datetime(holi.locdate,format="%Y%m%d")

In [239]:
# sub2.holi는 공휴일 -> 주말은 아님
sub2["holi"]=sub2["날짜"].isin(holi.locdate)

In [240]:
sub2[sub2["year"]==2017][sub2.holi==True]
# 1900개행이 아닌 1800개 --> 20171003이 개천절과 추석이 겹쳐있기 때문에 중복!

,날짜,역명,구분,year,month,요일,17to20,holi
73100,2017-01-01,시청,승차,2017,1,일,1901,True
73101,2017-01-01,시청,하차,2017,1,일,958,True
73102,2017-01-01,을지로입구,승차,2017,1,일,7001,True
73103,2017-01-01,을지로입구,하차,2017,1,일,4080,True
73104,2017-01-01,을지로3가,승차,2017,1,일,1772,True
...,...,...,...,...,...,...,...,...
108995,2017-12-25,양천구청,하차,2017,12,월,1073,True
108996,2017-12-25,신정네거리,승차,2017,12,월,861,True
108997,2017-12-25,신정네거리,하차,2017,12,월,1727,True
108998,2017-12-25,용두,승차,2017,12,월,264,True


In [241]:
# 주말이거나 공휴일이면 holi가 True가 되도록 변경
sub2["holi"]=np.where((sub2["요일"].isin(["토","일"]))|(sub2["holi"]==True),True,False)

In [242]:
sub2[sub2.holi==True].head(60)

,날짜,역명,구분,year,month,요일,17to20,holi
0,2015-01-01,시청,승차,2015,1,목,2381,True
1,2015-01-01,을지로입구,승차,2015,1,목,6598,True
2,2015-01-01,을지로3가,승차,2015,1,목,1150,True
3,2015-01-01,을지로4가,승차,2015,1,목,665,True
4,2015-01-01,동대문역사문화공원,승차,2015,1,목,2895,True
5,2015-01-01,신당,승차,2015,1,목,1270,True
6,2015-01-01,상왕십리,승차,2015,1,목,649,True
7,2015-01-01,왕십리,승차,2015,1,목,2157,True
8,2015-01-01,한양대,승차,2015,1,목,698,True
9,2015-01-01,뚝섬,승차,2015,1,목,953,True


```
csv파일 생성 완료라 markdown으로 바꿔놓음
sub2.to_csv("sub2.csv",index=False,encoding="euc-kr")
```

## 신설동라인같은 역 제거 필요!!!

```





```

### 기상 데이터 전처리